<a href="https://colab.research.google.com/github/dtht2d/deep-learning/blob/main/codes/face-recognition/simple_face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install libraries


# **Python- Simple Face Recognition**
## Objective
Create a network that identifies each subject as a class (instead of the aforesaid example’s objects and their labels, here we have face images assigned to their unique identities). Use the same parameters as MATLAB’s example but also try to tune them for better results. Report on resulting validation accuracies similar to the aforesaid MATLAB example. 
Read MATLAB help files on “imageDatastore” and its various options. Repeat the above with a VGG 19 and compare results with AlexNet's. 

## Dataset
*   [the AT&T face recognition dataset](https://www.kaggle.com/datasets/kasikrit/att-database-of-faces)
  (40 subjects with 10 different mugshots for each).

  Follow MATLAB’s transfer learning example https://www.mathworks.com/help/deeplearning/examples/transfer-learning-using-alexnet.html

### Install libraries

In [13]:
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post4-py3-none-any.whl size=2973 sha256=cbf163ac6668e8e58a738a46a96d9edd2fc4b7961b38c2d2f0abb75432c40397
  Stored in directory: /root/.cache/pip/wheels/d5/b2/a9/590d15767d34955f20a9a033e8db973b79cb5672d95790c0a9
Successfully built sklearn


## Load Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
!unzip /content/drive/MyDrive/deep-learning/ATT/archive.zip -d /content/drive/MyDrive/deep-learning/ATT

Archive:  /content/drive/MyDrive/deep-learning/ATT/archive.zip
  inflating: /content/drive/MyDrive/deep-learning/ATT/README  
  inflating: /content/drive/MyDrive/deep-learning/ATT/s1/1.pgm  
  inflating: /content/drive/MyDrive/deep-learning/ATT/s1/10.pgm  
  inflating: /content/drive/MyDrive/deep-learning/ATT/s1/2.pgm  
  inflating: /content/drive/MyDrive/deep-learning/ATT/s1/3.pgm  
  inflating: /content/drive/MyDrive/deep-learning/ATT/s1/4.pgm  
  inflating: /content/drive/MyDrive/deep-learning/ATT/s1/5.pgm  
  inflating: /content/drive/MyDrive/deep-learning/ATT/s1/6.pgm  
  inflating: /content/drive/MyDrive/deep-learning/ATT/s1/7.pgm  
  inflating: /content/drive/MyDrive/deep-learning/ATT/s1/8.pgm  
  inflating: /content/drive/MyDrive/deep-learning/ATT/s1/9.pgm  
  inflating: /content/drive/MyDrive/deep-learning/ATT/s10/1.pgm  
  inflating: /content/drive/MyDrive/deep-learning/ATT/s10/10.pgm  
  inflating: /content/drive/MyDrive/deep-learning/ATT/s10/2.pgm  
  inflating: /content/dr

In [21]:
path = "/content/drive/MyDrive/deep-learning/ATT"

**Function to get images with its class name**

In [48]:
import os
import itertools

def getIMAGElist(data_folder, n_class, n_image):
    """
    This function gets a list of images.
    data_folder: contains different class folders.
    n_class: number of class folders to include.
    n_image: number of images to include in each class folder.
    Returns:
        Image list, target name.
    """
    image_list = []
    target_name = []
    with os.scandir(data_folder) as entries:
        for entry in itertools.islice(entries, n_class):
            if not entry.name.startswith('.'):
                with os.scandir(entry.path) as files:
                    for file in itertools.islice(files, n_image):
                        if not file.name.startswith('.'):
                            image_path = os.path.join(data_folder, entry.name, file.name)
                            image_list.append(image_path)
                            target_name.append(entry.name)
    return image_list, target_name


In [50]:
data_folder = path
n_class = 40
n_image = 10
image_list, target_name = getIMAGElist(data_folder,n_class,n_image) 
print (len(image_list))
print (len(target_name))

400
400


In [52]:
import cv2
import numpy as np 
data_folder = path
n_class = 40
n_image = 10
image_list, class_name= getIMAGElist(data_folder,n_class,n_image)
image_data=[]
for i in range(len(image_list)):
    image = cv2.imread(image_list[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Resize image: INTER_AREA-resampling using pixel area relation
    image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
    image = np.array(image)
    image = image.astype('float32')
    image /= 255
    image_data.append(image)
class_dict={k:v for v,k in enumerate(np.unique(class_name))}
class_val= [class_dict[class_name[i]] for i in range(len(class_name))]
print (len(image_data))
print (class_dict)
print (len(class_val))

400
{'s1': 0, 's10': 1, 's11': 2, 's12': 3, 's13': 4, 's14': 5, 's15': 6, 's16': 7, 's17': 8, 's18': 9, 's19': 10, 's2': 11, 's20': 12, 's21': 13, 's22': 14, 's23': 15, 's24': 16, 's25': 17, 's26': 18, 's27': 19, 's28': 20, 's29': 21, 's3': 22, 's30': 23, 's31': 24, 's32': 25, 's33': 26, 's34': 27, 's35': 28, 's36': 29, 's37': 30, 's38': 31, 's39': 32, 's4': 33, 's40': 34, 's5': 35, 's6': 36, 's7': 37, 's8': 38, 's9': 39}
400


## Split dataset 
Train:Test:Val = 60:20:20 so test and validation set have same ratio

In [53]:
from sklearn.model_selection import train_test_split
x = np.array(image_data)
y = np.array(class_val)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=40, stratify=y)
#0.2/0.8=0.25
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=40,  stratify=y_train)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_val.shape, y_val.shape)

(240, 224, 224, 3) (240,)
(80, 224, 224, 3) (80,)
(80, 224, 224, 3) (80,)


**Comment:**

`random_state `: set to a fixed value, we can ensure that the same split is obtained every time we run the code, which can be useful for reproducibility and debugging purposes.

` stratify `: set to y (or y_train), which means that the split is made in a way that preserves the proportion of each class in y ( or y
_train.